In [1]:
import numpy as np
import pandas as pd

In [2]:
customer_orders = pd.read_csv('customer_orders.csv')
runner_orders = pd.read_csv('runner_orders.csv')
pizza_names = pd.read_csv('pizza_names.csv')
pizza_recipes = pd.read_csv('pizza_recipes.csv')
pizza_toppings = pd.read_csv('pizza_toppings.csv')
runners = pd.read_csv('runners.csv')

In [3]:
# modify values of "distance", and "duration" columns
runner_orders["distance"] = runner_orders["distance"] = runner_orders["distance"].str.replace('km','').astype(float)
runner_orders['duration'] = runner_orders['duration'].str.replace('mins|minutes|minute', '').astype(float)

In [4]:
# unified data type of primary key, and foreign key
customer_orders['order_id'] = customer_orders['order_id'].astype(int)
customer_orders['pizza_id'] = customer_orders['pizza_id'].astype(int)
pizza_names['pizza_id'] = pizza_names['pizza_id'].astype(int)
pizza_toppings['topping_id'] = pizza_toppings['topping_id'].astype(int)

# Part I

Q1. How many pizzas were ordered?

In [5]:
customer_orders["order_id"].count()

14

Q2. How many unique customer orders was made?

In [6]:
customer_orders["order_id"].nunique()

10

Q3. How many successful orders was delivered by each runner?


In [7]:
# create source
successful_ordered = runner_orders['runner_id'].loc[runner_orders["cancellation"].isnull()].value_counts().reset_index()
successful_ordered.columns = ['runner_id', 'order_delivered']

# print result
successful_ordered

,runner_id,order_delivered
0,1,4
1,2,3
2,3,1


Q4. How many of each type of pizza was delivered?

In [8]:
# delivered orders
delivered_runners = runner_orders.loc[runner_orders['cancellation'].isnull()]

# merge customer orders, filter of delivered runners, and pizza names
pizza_delivered = (
    customer_orders.merge(delivered_runners, on='order_id', how='inner')
    .merge(pizza_names, on='pizza_id', how='left')
)

# count pizza types
pizza_delivered_counts = pizza_delivered['pizza_name'].value_counts().reset_index()
pizza_delivered_counts.columns = ['pizza_name', 'pizza_count']

# print result
pizza_delivered_counts

,pizza_name,pizza_count
0,Meatlovers,9
1,Vegetarian,3


Q5. How many Vegetarian and Meatlovers were ordered by each customer?

In [9]:
# create source
pizza_count_by_customer = (
    customer_orders.merge(pizza_names, on='pizza_id', how='left')
    .groupby(['customer_id', 'pizza_name'])
    .size()
    .reset_index(name='order_made')
    .sort_values('customer_id')
)

# print result
pizza_count_by_customer.sort_values('customer_id')

,customer_id,pizza_name,order_made
0,101,Meatlovers,2
1,101,Vegetarian,1
2,102,Meatlovers,2
3,102,Vegetarian,1
4,103,Meatlovers,3
5,103,Vegetarian,1
6,104,Meatlovers,3
7,105,Vegetarian,1


Q6. What was the maximum number of pizzas delivered in a single orders?

In [10]:
# create source
max_pizzas_delivered = pizza_delivered['order_id'].value_counts(ascending=False).reset_index(name='pizzas').head(1)
max_pizzas_delivered.columns = ['order_id', 'pizzas']

# print result
max_pizzas_delivered


,order_id,pizzas
0,4,3


Q7. For each customer, how many delivered pizzas had at least 1 change and how many had no changes?

In [11]:
# perform the inner join to get delivered orders
merged_data = customer_orders.merge(runner_orders[runner_orders['cancellation'].isnull()],
                                        on='order_id', how='inner')

# define conditions for sum calculations
merged_data['at_least_1_change'] = (
    merged_data['exclusions'].notnull() | merged_data['extras'].notnull()
).astype(int)

merged_data['no_change'] = (
    (merged_data['exclusions'].isnull() & merged_data['extras'].isnull())
).astype(int)

# group by 'customer_id' and perform the sums
result = merged_data.groupby('customer_id').agg({
    'at_least_1_change': 'sum',
    'no_change': 'sum'
}).reset_index().sort_values('customer_id')

# print the result
result

,customer_id,at_least_1_change,no_change
0,101,0,2
1,102,0,3
2,103,3,0
3,104,2,1
4,105,1,0


Q8. How many pizzas were delivered that had both exclusions and extras?

In [12]:
# define conditions for sum calculations
merged_data['change_extras_and_exclusion'] = (
    merged_data['exclusions'].notnull() & merged_data['extras'].notnull()
).astype(int)

# results
print(merged_data['change_extras_and_exclusion'].sum())

1


Q9. What was the total volume of pizzas ordered for each hour of the day?

In [13]:
# change date type of oder_time column to pd.datetime
customer_orders['order_time'] = pd.to_datetime(customer_orders['order_time'], format='%d/%m/%Y %H:%M:%S')

# extract the hour of the date
customer_orders['hour_of_the_day'] = customer_orders['order_time'].dt.hour

# define measure
result = customer_orders.groupby('hour_of_the_day').size().reset_index(name='pizza_ordered_count')

# print order result
print(result.sort_values('hour_of_the_day'))

   hour_of_the_day  pizza_ordered_count
0               11                    1
1               13                    3
2               18                    3
3               19                    1
4               21                    3
5               23                    3


Q10. What was the volume of orders for each day of the week?

In [14]:
# extract day name of the week
customer_orders['day_of_week'] = customer_orders['order_time'].dt.day_name()

# define measure
result = customer_orders.groupby('day_of_week').size().reset_index(name='pizza_order_count')

# day of week order
d_o_w_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

result['day_of_week'] = pd.Categorical(result['day_of_week'], categories=d_o_w_order, ordered=True)

# order result
print(result.sort_values('day_of_week'))

  day_of_week  pizza_order_count
3   Wednesday                  5
2    Thursday                  3
0      Friday                  1
1    Saturday                  5


# Part II

Q1. How many runners signed up for each week period? (i.e. week starts 2021-01-01)

In [15]:
# format data type
runners['registration_date'] = pd.to_datetime(runners['registration_date'], format='%d/%M/%Y')

# define measure
runners['registration_week_year'] = (runners['registration_date'].dt.strftime('%W').astype(int) + 1).astype(str) + '-' + runners['registration_date'].dt.strftime('%Y')

# calculate result
result = runners.groupby('registration_week_year').size().reset_index(name='registered_runners')

# print result
print(result)

  registration_week_year  registered_runners
0                 1-2021                   2
1                 2-2021                   1
2                 3-2021                   1


Q2. What was the average time in minutes it took for each runner to arrive at the Pizza Runner HQ to pickup the order?

In [16]:
pizza_delivered['pickup_time'] = pd.to_datetime(pizza_delivered['pickup_time'], format = '%d/%m/%Y %H:%M:%S')
pizza_delivered['order_time'] = pd.to_datetime(pizza_delivered['order_time'], format = '%d/%m/%Y %H:%M:%S')
pizza_delivered['pickup_time_in_minutes'] = (pizza_delivered['pickup_time'] - pizza_delivered['order_time']).dt.total_seconds() / 60
pizza_delivered[['runner_id', 'pickup_time_in_minutes']].groupby(by='runner_id').agg({'pickup_time_in_minutes': 'mean'})

,pickup_time_in_minutes
runner_id,
1,15.677778
2,23.720000
3,10.466667


Q3. Is there any relationship between the number of pizzas and how long the order takes to prepare?

In [17]:
result = pizza_delivered.groupby(by='order_id').agg(
                                                pizzas_in_orders = ('pizza_id', 'count'),
                                                avg_pickup_time_in_minutes = ('pickup_time_in_minutes', 'mean'))
result['avg_time_per_pizza'] = result['avg_pickup_time_in_minutes']/result['pizzas_in_orders']
result = result.round({'avg_pickup_time_in_minutes': 2, 'avg_time_per_pizza': 2})
result

,pizzas_in_orders,avg_pickup_time_in_minutes,avg_time_per_pizza
order_id,,,
1,1,10.53,10.53
2,1,10.03,10.03
3,2,21.23,10.62
4,3,29.28,9.76
5,1,10.47,10.47
7,1,10.27,10.27
8,1,20.48,20.48
10,2,15.52,7.76


Q4. What was the average distance travelled for each customer?

In [19]:
result = pizza_delivered.groupby()

,order_id,customer_id,pizza_id,exclusions,extras,order_time,runner_id,pickup_time,distance,duration,cancellation,pizza_name,pickup_time_in_minutes
0,1,101,1,NaN,NaN,2020-01-01 18:05:02,1,2020-01-01 18:15:34,20.0,32.0,NaN,Meatlovers,10.533333
1,2,101,1,NaN,NaN,2020-01-01 19:00:52,1,2020-01-01 19:10:54,20.0,27.0,NaN,Meatlovers,10.033333
2,3,102,1,NaN,NaN,2020-01-02 23:51:23,1,2020-01-03 00:12:37,13.4,20.0,NaN,Meatlovers,21.233333
3,3,102,2,NaN,NaN,2020-01-02 23:51:23,1,2020-01-03 00:12:37,13.4,20.0,NaN,Vegetarian,21.233333
4,4,103,1,4,NaN,2020-01-04 13:23:46,2,2020-01-04 13:53:03,23.4,40.0,NaN,Meatlovers,29.283333
